<h1>Efficient Portfolio Construction</h1>
<h4>Blake Rayvid - <a href=https://github.com/brayvid>https://github.com/brayvid</a></h4>

**Note:** You must be logged into Google to run this notebook. In the event the file `sp100_returns_10y.csv` is not already available, you would need to download it [here](https://github.com/brayvid/efficient-portfolio/blob/main/example/sp100_returns_10y.csv) (GitHub) and upload it to the notebook by clicking the folder button on the left and then the upload button. The base directory (which includes the folder `sample_data`) is typically `/content/` unless you connect your own cloud storage, in which case you must update the variable `dir` below to match your setup.

### Description

In *An Analytic Derivation of the Efficient Portfolio Frontier* (1972), [Robert C. Merton](https://en.wikipedia.org/wiki/Robert_C._Merton) described an algorithm to construct a portfolio with the <ins>smallest variance in expected returns for a given level of expected returns</ins> (efficient). The algorithm takes as input a list of candidate securities and their historical annual returns as decimal values, and it outputs the fraction of the portfolio to be allocated to each. Each of these output percentages may be positive or negative corresponding to long and short positions, or zero when no position should be taken, but they will sum to 100%. The example below generates an allocation strategy using the whole S&P 100.

### Inputs

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Algorithm/program inputs

# Provide a decimal value for expected annual returns (100% = 1.0)
E = 0.25

dir = '/content/'

# Provide name of existing returns csv file
rets = "sp100_returns_10y.csv"

# Provide an ID string for the output file
outString = "sp100_alloc_10y"

### Merton's algorithm
[Read the paper](http://www.stat.ucla.edu/~nchristo/statistics_c183_c283/analytic_derivation_frontier.pdf)

In [ ]:
# Intermediate coeff A
def A(Ei, Vij):
  m = len(Ei)
  outerS = 0
  for k in range(m):
    innerS = 0
    for j in range(m):
      innerS += Vij[k][j] * Ei[j]
    outerS += innerS
  return outerS

# Intermediate coeff B
def B(Ei, Vij):
  m = len(Ei)
  outerS = 0
  for k in range(m):
    innerS = 0
    for j in range(m):
      innerS += Vij[k][j] * Ei[j] * Ei[k]
    outerS += innerS
  return outerS

# Intermediate coeff C
def C(Ei, Vij):
  m = len(Ei)
  outerS = 0
  for k in range(m):
    innerS = 0
    for j in range(m):
      innerS += Vij[k][j]
    outerS += innerS
  return outerS

# Intermediate coeff D
def D(a, b, c):
  return b * c - a * a

# Calculation of each Xi
def Xk(k, a, b, c, d, e, Vij, Ei):
  m = len(Ei)
  Lsum = 0
  for j in range(m):
    Lsum += (Vij[k][j] * (c * Ei[j] - a))
  Rsum = 0
  for j in range(m):
    Rsum += (Vij[k][j] * (b -  a * Ei[j]))
  return (e * Lsum + Rsum) / d

def Merton(E, R):
  avg_ret = np.mean(R,axis=1)
  cov_mat = np.cov(R)
  inv_cov = np.linalg.inv(cov_mat)
  m = len(avg_ret)

  # Coefficients
  a = A(avg_ret, inv_cov)
  b = B(avg_ret, inv_cov)
  c = C(avg_ret, inv_cov)
  d = D(a,b,c)

  # Compute each Xi
  outputX = np.zeros(m)
  for k in range(m):
    outputX[k] = Xk(k, a, b, c, d, E, inv_cov, avg_ret)
  return outputX

### Runner code

In [ ]:
# This cell runs the algorithm and displays the results.

# # Historical returns data already generated so just read the csv file
data = pd.read_csv(dir + rets,index_col=0,header=0)

# Separate values from symbols
R = data.values
symbs = data.index

# Compute distribution
X = Merton(E, R)

# Print results
print('Assumed total annual return: '+ str(np.around(E*100,1)) + '%.\n\nMinimium-variance portfolio is:')
for ind, val in enumerate(X):
  print('X('+str(symbs[ind])+') =', str(np.around(100 * val)) + '%','(LONG)' if val > 0 else '(SHORT)')
s = np.around(sum(X) * 100)
rfx = 100 - s
print('\nSum of X =',str(s + rfx) + '%')

# Save CSV file
xdf = pd.DataFrame(data=X,index=symbs, columns=["Allocation"])
xdf.to_csv(dir + outString + '_' + 'E=' + str((E*100)) + '%.csv')

Assumed total annual return: 25.0%.

Minimium-variance portfolio is:
X(AAPL) = 12.0% (LONG)
X(ABT) = 4.0% (LONG)
X(ACN) = -9.0% (SHORT)
X(ADBE) = 1.0% (LONG)
X(AIG) = -1.0% (SHORT)
X(ALL) = 13.0% (LONG)
X(AMGN) = 2.0% (LONG)
X(AMT) = 38.0% (LONG)
X(AMZN) = -3.0% (SHORT)
X(AXP) = -20.0% (SHORT)
X(BA) = 11.0% (LONG)
X(BAC) = 5.0% (LONG)
X(BIIB) = -5.0% (SHORT)
X(BK) = -1.0% (SHORT)
X(BKNG) = 1.0% (LONG)
X(BLK) = 6.0% (LONG)
X(BMY) = 14.0% (LONG)
X(BRK-B) = 17.0% (LONG)
X(C) = 7.0% (LONG)
X(CAT) = -14.0% (SHORT)
X(CHTR) = -4.0% (SHORT)
X(CL) = -11.0% (SHORT)
X(CMCSA) = -1.0% (SHORT)
X(COF) = 4.0% (LONG)
X(COP) = -13.0% (SHORT)
X(COST) = -35.0% (SHORT)
X(CRM) = 2.0% (LONG)
X(CSCO) = -11.0% (SHORT)
X(CVS) = -2.0% (SHORT)
X(CVX) = 2.0% (LONG)
X(DD) = 3.0% (LONG)
X(DHR) = 17.0% (LONG)
X(DIS) = -17.0% (SHORT)
X(DUK) = 4.0% (LONG)
X(EMR) = 6.0% (LONG)
X(EXC) = 4.0% (LONG)
X(F) = -2.0% (SHORT)
X(FDX) = 3.0% (LONG)
X(GD) = -7.0% (SHORT)
X(GE) = -1.0% (SHORT)
X(GILD) = -2.0% (SHORT)
X(GM) = -4.0% 